In [1]:
import numpy as np
import pandas as pd
from os import listdir
import matplotlib.pyplot as plt
import itertools as it
from statsmodels.sandbox.stats.multicomp import multipletests
import sklearn.preprocessing as pp
import statsmodels.api as sm
#import nltk
import scipy.stats as st
import statsmodels.formula.api as smf
import seaborn as sns
import Helper as hp

#Load Data
data = pd.read_csv("CHI_2019_FULL.csv")

#Set Labels 
emotion_label = ['Anger', 'Boredom', 'Disgust', 'Fear', 'Happiness', 'Emo_Neutral', 'Sadness']
affect_label = ['Aggressiv', 'Cheerful', 'Intoxicated', 'Nervous', 'Aff_Neutral', 'Tired']
loi_label = ['Disinterest', 'Normal', 'High Interest']

#Get specific data and save it into new data frames
# We use the pandas .copy(deep=True) function to prevent the SettingWithCopyWarning we would otherwise get. Since we do
# not write, but only read from the data, the warning does not affect the data frames
df_emotion = data[['Anger', 'Boredom', 'Disgust', 'Fear', 'Happiness', 'Emo_Neutral', 'Sadness', 'Filename']].copy(deep=True)
df_affect = data[['Aggressiv', 'Cheerful', 'Intoxicated', 'Nervous', 'Aff_Neutral', 'Tired', 'Filename']].copy(deep=True)
df_loi = data[['Disinterest', 'Normal', 'High Interest', 'Filename']].copy(deep=True)
df_ar_val = data[['Arousal', 'Valence', 'Filename']].copy(deep=True)
#For further usage, we want to append the CharacterID as a column, which is saved with other information in the filename
#Since we only want the digits, we can remove all non-digit characters of the filename column and append the column to the df

df_emotion['Char_ID'] = df_emotion['Filename'].replace('\D+','', regex = True).copy(deep=True)
df_affect['Char_ID'] = df_affect['Filename'].replace('\D+','', regex = True).copy(deep=True)
df_loi['Char_ID'] = df_loi['Filename'].replace('\D+','', regex = True).copy(deep=True)
df_ar_val['Char_ID'] = df_ar_val['Filename'].replace('\D+','', regex = True).copy(deep=True)

#Load data about speakers
char_data = pd.read_csv("CHI_2019_CharacterData.csv")  

#Join above tables and Character Tables

#To Join DataFrames we have to cast the column on which we want to join to int, so that both columns have the same data type
char_data['ID'] = char_data['ID'].astype(int)
df_ar_val['Char_ID'] = df_ar_val['Char_ID'].astype(int)
df_emotion['Char_ID'] = df_emotion['Char_ID'].astype(int)
df_affect['Char_ID'] = df_affect['Char_ID'].astype(int)
df_loi['Char_ID'] = df_loi['Char_ID'].astype(int)

#Save new data frames
df_ar_val_char = df_ar_val.merge(char_data, how = 'left', left_on='Char_ID', right_on='ID')
df_emotion_char = df_emotion.merge(char_data, how = 'left', left_on='Char_ID', right_on= 'ID')
df_affect_char = df_affect.merge(char_data, how = 'left', left_on='Char_ID', right_on= 'ID')
df_loi_char = df_loi.merge(char_data, how = 'left', left_on='Char_ID', right_on= 'ID')

#Now, we only want to have data containing information about the answers
#For that we need to extract from the filename column, whether the file was part of an answer
#a = answer, p = presentation, q = question
#sentence_type should be the same for all tables, but just to be sure
arval_sentence_type = df_ar_val_char.Filename.str.replace('\d+','').str[3:-4]
df_ar_val_char['SentenceType'] = arval_sentence_type
emo_sentence_type = df_emotion_char.Filename.str.replace('\d+','').str[3:-4]
df_emotion_char['SentenceType'] = emo_sentence_type
aff_sentence_type = df_affect_char.Filename.str.replace('\d+','').str[3:-4]
df_affect_char['SentenceType'] = aff_sentence_type
loi_sentence_type = df_loi_char.Filename.str.replace('\d+','').str[3:-4]
df_loi_char['SentenceType'] = loi_sentence_type

#For affect, we will have to drop the intoxication column and thus we will re-normalize the other values
affect_label.remove('Intoxicated')
df_affect_char = df_affect_char.drop(['Intoxicated'], axis = 1)
norm_test = pp.normalize(df_affect_char[affect_label], norm = 'l1')
df_affect_char[affect_label] = norm_test

df_loi_char['Normal Interest'] = df_loi_char['Disinterest'] + df_loi_char['Normal']
df_loi_char = df_loi_char.drop(['Disinterest', 'Normal'], axis = 1)
loi_label = ['Normal Interest', 'High Interest']

C:\Users\user\Anaconda3\lib\site-packages\statsmodels\compat\pandas.py:49: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  data_klasses = (pandas.Series, pandas.DataFrame, pandas.Panel)


In [2]:
#Now select only those who have SentenceType == 'a'
df_ar_val_char_a = df_ar_val_char.loc[df_ar_val_char['SentenceType'] == 'a']
df_emotion_char_a = df_emotion_char.loc[df_emotion_char['SentenceType'] == 'a']
df_affect_char_a = df_affect_char.loc[df_affect_char['SentenceType'] == 'a']
df_loi_char_a = df_loi_char.loc[df_loi_char['SentenceType'] == 'a']

#Now select only those who have SentenceType == 'a'
df_ar_val_char_p = df_ar_val_char.loc[df_ar_val_char['SentenceType'] == 'p'].copy()
df_emotion_char_p = df_emotion_char.loc[df_emotion_char['SentenceType'] == 'p'].copy()
df_affect_char_p = df_affect_char.loc[df_affect_char['SentenceType'] == 'p'].copy()
df_loi_char_p = df_loi_char.loc[df_loi_char['SentenceType'] == 'p'].copy()

#Now select only those who have SentenceType == 'a'
df_ar_val_char_q = df_ar_val_char.loc[df_ar_val_char['SentenceType'] == 'q']
df_emotion_char_q = df_emotion_char.loc[df_emotion_char['SentenceType'] == 'q']
df_affect_char_q = df_affect_char.loc[df_affect_char['SentenceType'] == 'q']
df_loi_char_q = df_loi_char.loc[df_loi_char['SentenceType'] == 'q']
df_loi_char_a

High Interest     Filename  Char_ID   ID  \
0         0.982720  192_a_a.wav      192  192   
2         0.979165  192_b_a.wav      192  192   
4         0.997420  192_c_a.wav      192  192   
6         0.848274  192_d_a.wav      192  192   
11        0.341174  197_a_a.wav      197  197   
..             ...          ...      ...  ...   
617       0.973040  457_c_a.wav      457  457   
619       0.978778  457_d_a.wav      457  457   
620       0.904425  457_e_a.wav      457  457   
621       0.964798  457_f_a.wav      457  457   
628       0.466249  464_a_a.wav      464  464   

                                            VideoTitle                  Name  \
0    23 Ways to Nudge: Review of Technology-Mediate...           Ana Caraban   
2    23 Ways to Nudge: Review of Technology-Mediate...           Ana Caraban   
4    23 Ways to Nudge: Review of Technology-Mediate...           Ana Caraban   
6    23 Ways to Nudge: Review of Technology-Mediate...           Ana Caraban   
11   A Badge, Not a Barrier: Designing for- and Trh...         Caroline Pitt   
..                                                 ...                   ...   
617  Interferi: Gesture Sensing using On-Body Acous...      Yasha Iravantchi   
619  Interferi: Gesture Sensing using On-Body Acous...      Yasha Iravantchi   
620  Interferi: Gesture Sensing using On-Body Acous...      Yasha Iravantchi   
621  Interferi: Gesture Sensing using On-Body Acous...      Yasha Iravantchi   
628  Interpreting the Diversity in Subjective Judgm...  Jean-Bernard Martens   

        Sex Academic Status      VideoID    IsNativeSpeaker SentenceType  \
0    Female    Grad Student  -U3Bo7KR108  Europ. Non-Native            a   
2    Female    Grad Student  -U3Bo7KR108  Europ. Non-Native            a   
4    Female    Grad Student  -U3Bo7KR108  Europ. Non-Native            a   
6    Female    Grad Student  -U3Bo7KR108  Europ. Non-Native            a   
11   Female    Grad Student  HxMSkH5ZpZI     Native Speaker            a   
..      ...             ...          ...                ...          ...   
617    Male    Grad Student  xBpBBpQh7GI     Native Speaker            a   
619    Male    Grad Student  xBpBBpQh7GI     Native Speaker            a   
620    Male    Grad Student  xBpBBpQh7GI     Native Speaker            a   
621    Male    Grad Student  xBpBBpQh7GI     Native Speaker            a   
628    Male             PhD  pS7CzHfawDU  Europ. Non-Native            a   

     Normal Interest  
0           0.017280  
2           0.020835  
4           0.002579  
6           0.151726  
11          0.658826  
..               ...  
617         0.026960  
619         0.021222  
620         0.095575  
621         0.035202  
628         0.533752  

[195 rows x 12 columns]

In [3]:
IDs = df_emotion_char_p['Char_ID'].copy()
IDs.drop_duplicates()

# For presentations we need an aggregate, e.g. median for the presenters
for i in IDs:
    #first for emotion
    for l in emotion_label:    
        emo = df_emotion_char_p.loc[df_emotion_char_p.Char_ID == i,l]
        emo_neu = hp.constructMedianSeries(emo)
        df_emotion_char_p.loc[df_emotion_char_p.Char_ID == i,l] = emo_neu
    #Then for affect
    for l in affect_label:    
        aff = df_affect_char_p.loc[df_affect_char_p.Char_ID == i,l]
        aff_neu = hp.constructMedianSeries(aff)
        df_affect_char_p.loc[df_affect_char_p.Char_ID == i,l] = aff_neu  
    for l in loi_label:    
        loi = df_loi_char_p.loc[df_loi_char_p.Char_ID == i,l]
        loi_neu = hp.constructMedianSeries(loi)
        df_loi_char_p.loc[df_loi_char_p.Char_ID == i,l] = loi_neu  
    for l in ['Arousal', 'Valence']:    
        arval = df_ar_val_char_p.loc[df_ar_val_char_p.Char_ID == i,l]
        arval_neu = hp.constructMedianSeries(arval)
        df_ar_val_char_p.loc[df_ar_val_char_p.Char_ID == i,l] = arval_neu 
        
#Now let's drop the duplicate values, since we only need one row now per person
df_emotion_char_p.drop_duplicates(subset=['ID'], inplace = True)
df_affect_char_p.drop_duplicates(subset=['ID'], inplace = True)
df_loi_char_p.drop_duplicates(subset=['ID'], inplace = True)
df_ar_val_char_p.drop_duplicates(subset=['ID'], inplace = True) 

In [4]:
emotion_label.extend(['Sex', 'Academic Status', 'IsNativeSpeaker'])
affect_label.extend(['Sex', 'Academic Status', 'IsNativeSpeaker'])
loi_label.extend(['Sex', 'Academic Status', 'IsNativeSpeaker'])

df_emotion_char_p.columns = df_emotion_char_p.columns.map(lambda x: 'P_' + x if x in emotion_label else x)
df_affect_char_p.columns = df_affect_char_p.columns.map(lambda x: 'P_' + x if x in affect_label else x)
df_loi_char_p.columns = df_loi_char_p.columns.map(lambda x: 'P_' + x if x in loi_label else x)
df_ar_val_char_p.columns = df_ar_val_char_p.columns.map(lambda x: 'P_' + x if x in ['Arousal', 'Valence', 'Sex', 'Academic Status', 'IsNativeSpeaker'] else x)

df_emotion_char_a.columns = df_emotion_char_a.columns.map(lambda x: 'A_' + x if x in emotion_label else x)
df_affect_char_a.columns = df_affect_char_a.columns.map(lambda x: 'A_' + x if x in affect_label else x)
df_loi_char_a.columns = df_loi_char_a.columns.map(lambda x: 'A_' + x if x in loi_label else x)
df_ar_val_char_a.columns = df_ar_val_char_a.columns.map(lambda x: 'A_' + x if x in ['Arousal', 'Valence', 'Sex', 'Academic Status', 'IsNativeSpeaker'] else x)

df_emotion_char_q.columns = df_emotion_char_q.columns.map(lambda x: 'Q_' + x if x in emotion_label else x)
df_affect_char_q.columns = df_affect_char_q.columns.map(lambda x: 'Q_' + x if x in affect_label else x)
df_loi_char_q.columns = df_loi_char_q.columns.map(lambda x: 'Q_' + x if x in loi_label else x)
df_ar_val_char_q.columns = df_ar_val_char_q.columns.map(lambda x: 'Q_' + x if x in ['Arousal', 'Valence', 'Sex', 'Academic Status', 'IsNativeSpeaker'] else x)

df_emotion_char_p = df_emotion_char_p.drop(['Name', 'Char_ID', 'VideoTitle', 'SentenceType'], axis = 1)
df_affect_char_p = df_affect_char_p.drop(['Name', 'Char_ID', 'VideoTitle', 'SentenceType'], axis = 1)
df_loi_char_p = df_loi_char_p.drop(['Name', 'Char_ID', 'VideoTitle', 'SentenceType'], axis = 1)
df_ar_val_char_p = df_ar_val_char_p.drop(['Name', 'Char_ID', 'VideoTitle', 'SentenceType'], axis = 1)

df_emotion_char_a = df_emotion_char_a.drop(['Name', 'Char_ID', 'VideoTitle', 'SentenceType'], axis = 1)
df_affect_char_a = df_affect_char_a.drop(['Name', 'Char_ID', 'VideoTitle', 'SentenceType'], axis = 1)
df_loi_char_a = df_loi_char_a.drop(['Name', 'Char_ID', 'VideoTitle', 'SentenceType'], axis = 1)
df_ar_val_char_a = df_ar_val_char_a.drop(['Name', 'Char_ID', 'VideoTitle', 'SentenceType'], axis = 1)

df_emotion_char_q = df_emotion_char_q.drop(['Name', 'Char_ID', 'VideoTitle', 'SentenceType'], axis = 1)
df_affect_char_q = df_affect_char_q.drop(['Name', 'Char_ID', 'VideoTitle', 'SentenceType'], axis = 1)
df_loi_char_q = df_loi_char_q.drop(['Name', 'Char_ID', 'VideoTitle', 'SentenceType'], axis = 1)
df_ar_val_char_q = df_ar_val_char_q.drop(['Name', 'Char_ID', 'VideoTitle', 'SentenceType'], axis = 1)

In [5]:
#To do: Merge Data Frames; First Question and Presentation, then answer and question
ar_val_char = df_ar_val_char_q.merge(df_ar_val_char_p, how = 'left', left_on='VideoID', right_on='VideoID')
loi_char = df_loi_char_q.merge(df_loi_char_p, how = 'left', left_on='VideoID', right_on='VideoID')
affect_char = df_affect_char_q.merge(df_affect_char_p, how = 'left', left_on='VideoID', right_on='VideoID')
emotion_char = df_emotion_char_q.merge(df_emotion_char_p, how = 'left', left_on='VideoID', right_on='VideoID')

In [6]:
df_ar_val_char_a.reset_index(drop = True, inplace = True)
df_ar_val_char_q.reset_index(drop = True, inplace = True)
arval = pd.concat([ar_val_char, df_ar_val_char_a], axis = 1)

df_loi_char_a.reset_index(drop = True, inplace = True)
df_loi_char_q.reset_index(drop = True, inplace = True)
loi = pd.concat([loi_char, df_loi_char_a], axis = 1)

df_affect_char_a.reset_index(drop = True, inplace = True)
df_affect_char_q.reset_index(drop = True, inplace = True)
affect = pd.concat([affect_char, df_affect_char_a], axis = 1)

df_emotion_char_a.reset_index(drop = True, inplace = True)
df_emotion_char_q.reset_index(drop = True, inplace = True)
emotion = pd.concat([emotion_char, df_emotion_char_a], axis = 1)